In [ ]:
import csv
from pykafka import KafkaClient
import json
from time import sleep
import threading
from datetime import datetime

# Configura Kafka
KAFKA_BROKER = "localhost:9091"  # Sostituisci con il tuo broker Kafka
client = KafkaClient(hosts=KAFKA_BROKER)

# Specifica l'intervallo di righe da leggere
START_ROW = 1  # Riga iniziale (0-indicizzata)
END_ROW = 620402   # Riga finale esclusa (0-indicizzata)

# Connettiti al broker Kafka
def invia_messaggi_a_kafka(file_path, start_row, end_row, topic_name):
    """
    Funzione per leggere un file CSV e inviare un intervallo di righe a Kafka.
    """
    topic = client.topics[topic_name]   
    producer = topic.get_sync_producer()

    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        for i, row in enumerate(reader):
            if start_row <= i < end_row:
                # Aggiorna il campo 'time' con il tempo corrente nel formato richiesto
                row['time'] = datetime.now().strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'
                
                messaggio = json.dumps(row).encode('utf-8')
                producer.produce(messaggio)
                sleep(0.049)



# Elenco dei file CSV
file_paths = [
    "orderedData/ITA.csv","orderedData/USA.csv"
]
topics = ["ITA","USA"]

# Crea e avvia i thread
threads = []
for file_path, topic in zip(file_paths, topics):

    thread = threading.Thread(target=invia_messaggi_a_kafka, 
        args=(file_path, START_ROW, END_ROW, "boat_data"))

    threads.append(thread)
    thread.start()

# Aspetta che tutti i thread terminino
for thread in threads:
    thread.join()

print("Tutti i file sono stati processati.")

Exception in thread Thread-6 (invia_messaggi_a_kafka):
Traceback (most recent call last):
  File "/Users/isaia/.pyenv/versions/3.10.14/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/Users/isaia/.pyenv/versions/3.10.14/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Users/isaia/.pyenv/versions/3.10.14/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/hb/dw89xtb95cl_b0blt4q7vzsr0000gn/T/ipykernel_3826/1798403710.py", line 31, in invia_messaggi_a_kafka
Exception in thread Thread-7 (invia_messaggi_a_kafka):
Traceback (most recent call last):
  File "/Users/isaia/.pyenv/versions/3.10.14/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
  File "/Users/isaia/.pyenv/versions/3.10.14/lib/python3.10/site-packages/pykafka/producer.py", line 411, in produce
    self.run()
  File "/Users/isaia/.pyenv/versions/3.10.14/

Tutti i file sono stati processati.


In [3]:
file_path = "orderedData/GBR.csv"
with open(file_path, 'r') as f:
    row_count = sum(1 for _ in f) - 1  # subtract 1 for header
print(f"Number of rows in {file_path}: {row_count}")

Number of rows in orderedData/GBR.csv: 1362997


In [4]:
with open(file_path, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    foiling_values = set()
    for row in reader:
        foiling_values.add(row.get('leg'))
print(f"Valori diversi nella colonna 'foiling': {foiling_values}")
print(f"Numero di valori diversi: {len(foiling_values)}")

Valori diversi nella colonna 'foiling': {'', 'up5', 'up3', 'up4', 'down1', 'down4', 'up1', 'up2', 'down3', 'postrace', 'prestart', 'down2'}
Numero di valori diversi: 12


In [5]:
def invia_messaggi_a_kafka(file_path, start_row, end_row, topic_name):
    """
    Funzione per leggere un file CSV e inviare un intervallo di righe a Kafka.
    """
    topic = client.topics[topic_name]   
    # Configura il produttore Kafka
    producer = topic.get_sync_producer()

    # Leggi il file CSV e invia l'intervallo di righe
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)  # Legge il file CSV come dizionari
        for i, row in enumerate(reader):
            if start_row <= i < end_row:
                messaggio = json.dumps(row).encode('utf-8')  # Serializza la riga in formato JSON
                producer.produce(messaggio)  # Invia il messaggio a Kafka
                sleep(0.05)
                #print(f"[{file_path}] Inviato: {messaggio}")